# Building a Volume at Price Chart

Volume-at-price is a chart used by technical analysts as a visual gauge to where the trading levels are concentrated, relative to price and volume.  In short, it is the sum of volume at different price levels.  This notebook utilizes individual components of the OpenBB SDK and combines them to build a new view.

In [67]:
# Import statements

from datetime import datetime, timedelta
from typing import Optional, Union
from openbb_terminal.sdk import openbb
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from openbb_terminal import OpenBBFigure
from openbb_terminal.common.technical_analysis import ta_helpers
from openbb_terminal.common.technical_analysis.overlap_model import sma, vwap
from openbb_terminal.rich_config import console


Let's look at the pieces we want to put together.

- Price
- Volume
- VWAP
- SMA

In [71]:
# Get the OHLC+V data

ticker= "QQQ"
ohlc_data = openbb.stocks.load(ticker, start_date = datetime.now().date().strftime("%Y-%m-%d"), interval=1, verbose=False)
ohlc_data.head(5)

,Open,High,Low,Close,Adj Close,Volume
date,,,,,,
2023-08-23 09:30:00,364.859985,364.859985,364.850006,364.850006,364.850006,1978013
2023-08-23 09:31:00,364.859985,365.269989,364.730011,365.130005,365.130005,197931
2023-08-23 09:32:00,365.119995,365.265015,364.640015,364.709900,364.709900,156792
2023-08-23 09:33:00,364.690002,365.156189,364.690002,364.959991,364.959991,133015
2023-08-23 09:34:00,364.970001,365.149292,364.750000,364.880005,364.880005,203328


Rounding the price levels - here to the nearest ten cents - will make the final chart easier display better.

In [76]:
# Separate volume and close into a new DataFrame and group by the price.

_vap = pd.DataFrame(data = [(round(ohlc_data["Close"], 1)),ohlc_data["Volume"]]).transpose()
_vap_df = _vap.groupby("Close").sum()[["Volume"]]
_vap_df.head(5)

,Volume
Close,
364.6,418695.0
364.7,156792.0
364.8,647207.0
364.9,2341760.0
365.0,261823.0


In [77]:
# Get VWAP levels

_vwap = vwap(ohlc_data)
_vwap.head(5)

,VWAP_D
date,
2023-08-23 09:30:00,364.853333
2023-08-23 09:31:00,364.870616
2023-08-23 09:32:00,364.870685
2023-08-23 09:33:00,364.874176
2023-08-23 09:34:00,364.878156


In [78]:
# Calculate the simple moving average.

_sma_df = sma(data = ohlc_data["Close"], length = 10)
_sma_df.tail(5)

,SMA_10
date,
2023-08-23 14:49:00,369.148868
2023-08-23 14:50:00,369.159866
2023-08-23 14:51:00,369.165869
2023-08-23 14:52:00,369.200879
2023-08-23 14:53:00,369.228598


## Parameterize the Components as a Function

To combine these components, a function needs to be created that parameterizes the inputs.

The two functions below represent the "model" and "view" components.  As Terminal components, they would be added to `openbb_terminal/common/technical_analysis/volume_model.py` and `volume_view.py` respectively. 

At the SDK level, users can call on either one, but only the view will return a chart and provide an input for the simple moving average.  As a Terminal as a function, the controller module would call the view function, `display_vap()`.

In [84]:
def vap(data: pd.DataFrame) -> pd.DataFrame:
    """Volume at Price.

    Parameters
    ----------
    data: pd.DataFrame
        Dataframe of OHLC + Volume.

    Returns
    -------
    pd.DataFrame
        Dataframe with technical indicator.
    """
    close_col = ta_helpers.check_columns(data, high=False, low=False)
    if close_col is None:
        return pd.DataFrame()
    if "Volume" not in data.columns:
        console.print("[red]Volume column not found[/red]\n")
        return pd.DataFrame()

    vap_df = pd.DataFrame(data=[(round(data["Close"], 1)), data["Volume"]]).transpose()

    return vap_df.groupby("Close").sum()[["Volume"]]

def display_vap(
    data: Optional[pd.DataFrame] = None,
    symbol: str = "",
    ma: Optional[int] = None,
    export: str = "",
    sheet_name: Optional[str] = None,
    external_axes: bool = False,
) -> Union[OpenBBFigure, None]:
    """Plots Volume-at-Price with an optional simple moving average.

    Parameters
    ----------
    data : pd.DataFrame
        Dataframe of ohlc prices
    symbol : str
        Ticker
    ma: int
        Window for calculating the simple moving average.
    sheet_name: str
        Optionally specify the name of the sheet the data is exported to.
    export: str
        Format to export data as
    external_axes : bool, optional
        Whether to return the figure object or not, by default False
    """
    if data is None or data.empty:
        data = openbb.stocks.load(symbol, start_date=datetime.now().strftime("%Y-%m-%d"), interval=1)
        if data.empty:
            return  None
    
    if ma is None:
        ma = 0
        ma_df = pd.DataFrame()

    if "vw" in data.columns:
        data["VWAP"] = data["vw"]
    data["VWAP"] = vwap(data)

    vap_df = vap(data)

    fig = make_subplots(
        rows=1, cols=2, shared_xaxes=False, shared_yaxes=True, horizontal_spacing=0.001
    )
    fig.add_trace(
        go.Bar(
            x=vap_df.Volume.astype(int),
            y=vap_df.index,
            orientation="h",
            name="Volume at Price",
            opacity=0.25,
            marker={"color": "gray"},
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(x=data.index, y=data["VWAP"], name="VWAP", marker={"color": "red"}),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=data.index, y=data["Close"], name="Close", marker={"color": "green"}
        ),
        row=1,
        col=2,
    )
    if ma != 0:
        ma_df = sma(data["Close"], ma).iloc[:, 0]
        fig.add_trace(
            go.Scatter(
                x=ma_df.index,
                y=ma_df,
                name=f"SMA {ma}",
                marker={"color": "orange"},
            ),
            row=1,
            col=2,
        )

    fig.update_layout(
        yaxis2=dict(
            zeroline=False,
            showticklabels=False,
            showgrid=False,
            mirror=True,
            showline=True,
            ticklen=0,
        ),
        xaxis2=dict(
            zeroline=False,
            mirror=True,
            showline=True,
            ticks="outside",
            showgrid=False,
            ticklen=0,
            domain=[0, 0],
        ),
        xaxis1=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=False,
            autorange=False,
            mirror=False,
            range=[0, max(vap_df.Volume)],
            ticklen=0,
            domain=[0, 0],
        ),
        yaxis1=dict(
            zeroline=False,
            showgrid=False,
            showticklabels=True,
            ticks="outside",
            mirror=True,
            showline=False,
            ticklen=0,
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        title=f"{symbol} - Volume at Price",
        title_y=1,
        title_x=0.5,
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.115,
            xanchor="right",
            x=1,
            bgcolor="rgba(0,0,0,0)"
        ),
    )
    # Using the OpenBBFigure class will provide additional processing like removing the time periods between trading days.
    ta = OpenBBFigure(fig)

    return ta.show()

The function will check if the `data` parameter is used, and will grab one-minute data from today if not.  There is also an optional parameter to add a simple moving average line.

In [80]:
display_vap(symbol="QQQ", ma=90)

Loading Intraday 1min data for QQQ with starting period 2023-08-23.

Alternatively, if the `data` parameter is fed a time series DataFrame, it can offer more flexibility.  Now the `symbol` parameter is just a title.  This mimics the behaviour of similar functions in the Terminal.

To see the difference between outputs of the Plotly figure object and the OpenBBFigure object, modify the last two lines in the `display_vap` function.  Comment out `ta=OpenBBFigure(fig)` and change `return ta.show()` to be `return fig.show()`.  You will notice that the gap between trading days is removed when the `OpenBBFigure` object is used.

Without OpenBBFigure:

```    
    # ta = OpenBBFigure(fig)

    return fig.show()
    ```

In [83]:
data = openbb.stocks.load("QQQ", start_date=(datetime.now()-timedelta(days=1)).date().strftime("%Y-%m-%d"), interval=15)
display_vap(data, symbol="QQQ - 15 Minute")

Loading Intraday 15min data for QQQ with starting period 2023-08-22.

With  OpenBBFigure:

```    
    ta = OpenBBFigure(fig)

    return ta.show()
    ```

In [85]:
data = openbb.stocks.load("QQQ", start_date=(datetime.now()-timedelta(days=1)).date().strftime("%Y-%m-%d"), interval=15)
display_vap(data=data, symbol="QQQ - 15 Minute")

Loading Intraday 15min data for QQQ with starting period 2023-08-22.